In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
#数据表介绍
#data_total_R20181212.csv #原始上限量明细数据
#ride_data.csv #乘联会数据
#work_days_data.csv #每个月工作日天数
#internet_data.csv #互联网数据
#festival_days_data.csv #节日数据

In [3]:
## 上限量数据
df=pd.read_csv('data_total_R20181212.csv',sep=',',encoding='gbk' )
df.columns=['year','month','vendor','brand','model','body_form','country','market_large','market_medium','market_small','fuel_type','y','type']
base_data=pd.DataFrame(df.groupby(['year','month'])['y'].sum())

#车类型数据(合资、自主、豪华 原始数据+百分比)
base_data_type=pd.DataFrame(df.groupby(['year','month','type'])['y'].sum()).unstack()
base_data=base_data.merge(base_data_type,how='left',on=['year','month'])  #加入乘联会数据
base_data.columns=['y','y_joint','y_independent','y_luxury']
base_data['y_joint_percent']=base_data.y_joint/base_data.y
base_data['y_independent_percent']=base_data.y_independent/base_data.y
base_data['y_luxury_percent']=base_data.y_luxury/base_data.y

#乘联会数据
df_ride=pd.read_csv('ride_data.csv',sep=',',encoding='gbk')
df_ride.columns=['year','month','y_ride']
base_data=base_data.merge(df_ride,how='left',on=['year','month'])  #加入乘联会数据

#合成日期数据
base_data['day']=1
base_data['ds'] = pd.to_datetime(base_data[['year', 'month', 'day']])

#加入每月天数
month_days={1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
base_data['days']=base_data['month'].map(month_days) 
base_data['days']=[29 if ((base_data.year[i]%4==0)&(base_data.month[i]==2)) else base_data.days[i] for i in base_data.index]
base_data.drop(['day'],axis=1,inplace=True)

#补充了几个月，后续原始数据更新过后可以不用操作
base_data.ds=[str(base_data.ds[i].strftime('%Y-%m-%d')) for i in range(base_data.shape[0])]
base_data.set_index('ds',inplace=True)
base_data.loc['2018-11-01'] = [2018,11,1874986,964466,666784,243736,964466/1874986,666784/1874986,243736/1874986,2020000,30]
base_data.loc['2018-12-01'] = [2018,12,2566147,1262767,1022719,280661,1262767/2566147,1022719/2566147,280661/2566147,2220000,31]
base_data.loc['2019-01-01'] = [2019,1,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,31]  #初始值设置不一样会有差异
base_data.loc['2019-02-01'] = [2019,2,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,28]  #初始值设置不一样会有差异

base_data=base_data[['year','month','days','y','y_joint','y_independent','y_luxury','y_joint_percent','y_independent_percent','y_luxury_percent']]

#每月工作日天数
df_work_data=pd.read_csv('work_days_data.csv',sep=',',encoding='gbk',index_col='ds')
base_data=base_data.merge(df_work_data,how='left',left_index=True,right_index=True)

#春节（前后各加两周共三周时间）占一二月份比例
df_festival_data=pd.read_csv('festival_days_data.csv',sep=',',encoding='gbk',index_col='ds')
#互联网数据
base_data.index=[pd.to_datetime(i).strftime('%Y-%m-%d') for i in base_data.index]
df_internet_data=pd.read_csv('internet_data.csv',sep=',',encoding='gbk',index_col='ds' )
base_data=base_data.merge(df_festival_data,how='left',left_index=True,right_index=True)
base_data=base_data.merge(df_internet_data,how='left',left_index=True,right_index=True)
base_data.festival_day_percent.fillna(0,inplace=True)
base_data.loc[:,list(df_internet_data.columns)]=base_data.loc[:,list(df_internet_data.columns)].fillna(0)

base_data.head()
base_data.to_csv('predict_data_final.csv')

,year,month,days,y,y_joint,y_independent,y_luxury,y_joint_percent,y_independent_percent,y_luxury_percent,work_days,festival_day_percent,order,intention,flow,retail,wholesale
2014-01-01,2014.0,1.0,31.0,2167589.0,1281845.0,716817.0,168927.0,0.591369,0.330698,0.077933,22.0,0.258065,0.0,0.0,0.0,0.0,0.0
2014-02-01,2014.0,2.0,28.0,803648.0,490241.0,239697.0,73710.0,0.610020,0.298261,0.091719,17.0,0.464286,0.0,0.0,0.0,0.0,0.0
2014-03-01,2014.0,3.0,31.0,1359944.0,870761.0,345412.0,143771.0,0.640292,0.253990,0.105718,21.0,0.000000,0.0,0.0,0.0,0.0,0.0
2014-04-01,2014.0,4.0,30.0,1455110.0,936349.0,370249.0,148512.0,0.643490,0.254447,0.102062,21.0,0.000000,0.0,0.0,0.0,0.0,0.0
2014-05-01,2014.0,5.0,31.0,1481658.0,942288.0,389955.0,149415.0,0.635969,0.263188,0.100843,21.0,0.000000,0.0,0.0,0.0,0.0,0.0
